In [1]:
# Import Required Libraries

import os
import numpy as np
import cv2
import face_detection 
from sklearn.cluster import DBSCAN
from keras.models import load_model
from keras.applications.resnet50 import preprocess_input
import tqdm
import argparse
# from imutils.video import FileVideoStream
# from imutils.video import FPS
# import imutils
import time
# from google.colab.patches import cv2_imshow
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:

"""
import cv2
import face_detection
print(face_detection.available_detectors)
detector = face_detection.build_detector("RetinaNetMobileNetV1", confidence_threshold=.5, nms_iou_threshold=.3)
mask_classifier = load_model("Models/mask_detector.model")

cap = cv2.VideoCapture(0)
c=0

while True:
    c+=1
    # BGR to RGB
    ret = cap.grab()
        
    if c%5==0:
        
        ret,img = cap.retrieve()

        if ret==False:
            break;



        # resize image
        # img = cv2.resize(img, (720, 720), interpolation = cv2.INTER_AREA)


        im=img[:, :, ::-1]
        detections = detector.detect(im)

        for face in detections:
            x1 =int(face[0])
            x2 =int(face[2])
            y1 =int(face[1])
            y2 =int(face[3])
            
            face_rgb = img[y1:y2,x1:x2,::-1]   
            
            # Preprocess the Image
            face_arr = cv2.resize(face_rgb, (224, 224), interpolation=cv2.INTER_NEAREST)
            face_arr = np.expand_dims(face_arr, axis=0)
            face_arr = preprocess_input(face_arr)
            # Predict if the Face is Masked or Not
            score = mask_classifier.predict(face_arr)

            if score[0][0]<0.2:
                cv2.rectangle(img, (x1, y1), (x2,y2), (0,0,255), 2)
                print("safe")
            else:
                cv2.rectangle(img, (x1, y1), (x2,y2), (0,255,0), 2)
                print("unsafe")
            

        cv2.imshow("Cam",img)
        # output_movie.write(img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break;
# Release Streams
cap.release()
cv2.destroyAllWindows()

# Good to Go!
print("Done !")


# Experiment 3
"""







'\nimport cv2\nimport face_detection\nprint(face_detection.available_detectors)\ndetector = face_detection.build_detector("RetinaNetMobileNetV1", confidence_threshold=.5, nms_iou_threshold=.3)\nmask_classifier = load_model("Models/mask_detector.model")\n\ncap = cv2.VideoCapture(0)\nc=0\n\nwhile True:\n    c+=1\n    # BGR to RGB\n    ret = cap.grab()\n        \n    if c%5==0:\n        \n        ret,img = cap.retrieve()\n\n        if ret==False:\n            break;\n\n\n\n        # resize image\n        # img = cv2.resize(img, (720, 720), interpolation = cv2.INTER_AREA)\n\n\n        im=img[:, :, ::-1]\n        detections = detector.detect(im)\n\n        for face in detections:\n            x1 =int(face[0])\n            x2 =int(face[2])\n            y1 =int(face[1])\n            y2 =int(face[3])\n            \n            face_rgb = img[y1:y2,x1:x2,::-1]   \n            \n            # Preprocess the Image\n            face_arr = cv2.resize(face_rgb, (224, 224), interpolation=cv2.INTER_NE

In [3]:
# Path to the Working Environment

# If using Google Colab (If on a Local Environment, no path required => set BASE_PATH  = "")
BASE_PATH = "drive/My Drive/Social_Distancing_with_AI-master/"

# Path to Input Video File in the BASE_PATH
FILE_PATH = "video1.mp4"
# Command-line input setup
"""
parser = argparse.ArgumentParser(description="SocialDistancing")
parser.add_argument("--videopath", type=str, default="video1.mp4", help="Path to the video file")
args = parser.parse_args()

FILE_PATH = args.videopath

"""


'\nparser = argparse.ArgumentParser(description="SocialDistancing")\nparser.add_argument("--videopath", type=str, default="video1.mp4", help="Path to the video file")\nargs = parser.parse_args()\n\nFILE_PATH = args.videopath\n\n'

In [4]:
# Initialize a Face Detector 
# Confidence Threshold can be Adjusted, Greater values would Detect only Clear Faces

print(face_detection.available_detectors)
detector = face_detection.build_detector("RetinaNetMobileNetV1", confidence_threshold=.5, nms_iou_threshold=.3)

['DSFDDetector', 'RetinaNetResNet50', 'RetinaNetMobileNetV1']


In [5]:
# Load Pretrained Face Mask Classfier (Keras Model)

mask_classifier = load_model("Models/face_mask_detector.model")

In [6]:
# Set the Safe Distance in Pixel Units (Minimum Distance Expected to be Maintained between People)
# This Parameter would Affect the Results, Adjust according to the Footage captured by CCTV Camera 

threshold_distance = 150  # Try with different Values before Finalizing
mask_detector_threshold = 0.5

In [7]:
##################################### Analyze the Video ################################################

# Load YOLOv3
net = cv2.dnn.readNet("Models/"+"yolov4.weights","Models/"+"yolov4.cfg")
# net = cv2.dnn.readNetFromCaffe("ssd/"+"MobileNet_deploy.prototxt","ssd/"+"MobileNet_deploy.caffemodel")

# Load COCO Classes
classes = []
with open("Models/"+"coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
    
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Fetch Video Properties
cap = cv2.VideoCapture(1)

# n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# Create Directory for Storing Results (Make sure it doesn't already exists !)
"""
os.mkdir(BASE_PATH+"Results")
os.mkdir(BASE_PATH+"Results/Extracted_Faces")
os.mkdir(BASE_PATH+"Results/Extracted_Persons")
os.mkdir(BASE_PATH+"Results/Frames")

"""



# Initialize Output Video Stream
"""

out_stream = cv2.VideoWriter(
    'Results/Output.mp4',
    cv2.VideoWriter_fourcc('X','V','I','D'),
    fps,
    (int(width),int(height)))
    
"""
c=0
print("Processing Frames :")
while cap.isOpened():
    
    ret = cap.grab()
    c+=1
    
    if c%5==0:
        print("frame start process",time.perf_counter())
        # Capture Frame-by-Frame
        ret,img = cap.retrieve()
        img = cv2.resize(img, (640, 480), interpolation = cv2.INTER_LINEAR) 

        # Check EOF
        if ret==False:
            break;

        # Get Frame Dimentions
        height, width, channels = img.shape
        # Detect Objects in the Frame with YOLOv3
        blob = cv2.dnn.blobFromImage(img, 0.00392, (256, 256), (0, 0, 0), True, crop=False)

        print("object detected",time.perf_counter())
        net.setInput(blob)
        outs = net.forward(output_layers)

        class_ids = []
        confidences = []
        boxes = []

        # Store Detected Objects with Labels, Bounding_Boxes and their Confidences
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:

                    # Get Center, Height and Width of the Box
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Topleft Co-ordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        # Initialize empty lists for storing Bounding Boxes of People and their Faces
        persons = []
        masked_faces = []
        unmasked_faces = []

        # Work on Detected Persons in the Frame
        for i in range(len(boxes)):
            if i in indexes:

                box = np.array(boxes[i])
                box = np.where(box<0,0,box)
                (x, y, w, h) = box

                label = str(classes[class_ids[i]])

                if label=='person':

                    persons.append([x,y,w,h])

                    # Save Image of Cropped Person (If not required, comment the command below)
                    """

                    cv2.imwrite(BASE_PATH + "Results/Extracted_Persons/"+str(frame)
                                +"_"+str(len(persons))+".jpg",
                                img[y:y+h,x:x+w])

                    """

                    # Detect Face in the Person
                    person_rgb = img[y:y+h,x:x+w,::-1]   # Crop & BGR to RGB
                    detections = detector.detect(person_rgb)

                    print("face detected",time.perf_counter())

                    # If a Face is Detected
                    if detections.shape[0] > 0:

                      detection = np.array(detections[0])
                      detection = np.where(detection<0,0,detection)

                      # Calculating Co-ordinates of the Detected Face
                      x1 = x + int(detection[0])
                      x2 = x + int(detection[2])
                      y1 = y + int(detection[1])
                      y2 = y + int(detection[3])

                      try :

                        # Crop & BGR to RGB
                        face_rgb = img[y1:y2,x1:x2,::-1]   

                        # Preprocess the Image
                        face_arr = cv2.resize(face_rgb, (224, 224), interpolation=cv2.INTER_NEAREST)
                        face_arr = np.expand_dims(face_arr, axis=0)
                        face_arr = preprocess_input(face_arr)

                        # Predict if the Face is Masked or Not
                        score = mask_classifier.predict(face_arr)

                        print("face classified",time.perf_counter())


                        # Determine and store Results
                        if score[0][0]<mask_detector_threshold:
                          unmasked_faces.append([x1,y1,x2,y2])
                        else:
                          masked_faces.append([x1,y1,x2,y2])

                        # Save Image of Cropped Face (If not required, comment the command below)
                        """
                        cv2.imwrite(BASE_PATH + "Results/Extracted_Faces/"+str(frame)
                                    +"_"+str(len(persons))+".jpg",
                                    img[y1:y2,x1:x2])
                        """

                      except:
                        continue


        # Calculate Coordinates of People Detected and find Clusters using DBSCAN
        person_coordinates = []

        for p in range(len(persons)):
          person_coordinates.append((persons[p][0]+int(persons[p][2]/2),persons[p][1]+int(persons[p][3]/2)))

        
        try:
            clustering = DBSCAN(eps=threshold_distance,min_samples=2).fit(person_coordinates)
            isSafe = clustering.labels_
        except:
            isSafe=0    
        

        # Count 
        person_count = len(persons)
        masked_face_count = len(masked_faces)
        unmasked_face_count = len(unmasked_faces)
        safe_count = np.sum((isSafe==-1)*1)
        unsafe_count = person_count - safe_count

        # Show Clusters using Red Lines
        arg_sorted = np.argsort(isSafe)

        for i in range(1,person_count):

          if isSafe[arg_sorted[i]]!=-1 and isSafe[arg_sorted[i]]==isSafe[arg_sorted[i-1]]:
            cv2.line(img,person_coordinates[arg_sorted[i]],person_coordinates[arg_sorted[i-1]],(0,0,255),2)

        # Put Bounding Boxes on People in the Frame
        for p in range(person_count):

          a,b,c,d = persons[p]

          # Green if Safe, Red if UnSafe
          if isSafe[p]==-1:
            cv2.rectangle(img, (a, b), (a + c, b + d), (0,255,0), 2)
          else:
            cv2.rectangle(img, (a, b), (a + c, b + d), (0,0,255), 2)

        # Put Bounding Boxes on Faces in the Frame
        # Green if Safe, Red if UnSafe
        for f in range(masked_face_count):

          a,b,c,d = masked_faces[f]
          cv2.rectangle(img, (a, b), (c,d), (0,255,0), 2)

        for f in range(unmasked_face_count):

          a,b,c,d = unmasked_faces[f]
          cv2.rectangle(img, (a, b), (c,d), (0,0,255), 2)

        # Show Monitoring Status in a Black Box at the Top
        cv2.rectangle(img,(0,0),(width,50),(0,0,0),-1)
        cv2.rectangle(img,(1,1),(width-1,50),(255,255,255),2)

        xpos = 15

        string = "Total People = "+str(person_count)
        cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
        xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]

        string = " ( "+str(safe_count) + " Safe "
        cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
        xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]

        string = str(unsafe_count)+ " Unsafe ) "
        cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
        xpos += cv2.getTextSize(string,cv2.FONT_HERSHEY_SIMPLEX,1,2)[0][0]

        string = "( " +str(masked_face_count)+" Masked "+str(unmasked_face_count)+" Unmasked "+\
                 str(person_count-masked_face_count-unmasked_face_count)+" Unknown )"
        cv2.putText(img,string,(xpos,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,255),2)

        # Write Frame to the Output File
        # out_stream.write(img)

        # Save the Frame in frame_no.png format (If not required, comment the command below)
        """
        cv2.imwrite(BASE_PATH+"Results/Frames/"+str(frame)+".jpg",img)

        """


        # Use if you want to see Results Frame by Frame
        cv2.imshow("Street Cam",img)
        # output_movie.write(img)

    # Exit on Pressing Q Key
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release Streams
cap.release()
cv2.destroyAllWindows()

# Good to Go!
print("Done !")

Processing Frames :
frame start process 52.0502802
object detected 52.0530713
face detected 81.5093354
face classified 87.4129036
frame start process 88.3372285
object detected 88.3399799
face detected 88.9334243
face classified 88.997671
frame start process 89.1223152
object detected 89.1248606
face detected 89.6823701
face classified 89.7412088
frame start process 89.8488126
object detected 89.8508394
face detected 90.4649883
face classified 90.5268451
frame start process 90.642274
object detected 90.6445611
face detected 91.2132789
face classified 91.2736497
frame start process 91.3782859
object detected 91.3805732
face detected 91.9710013
face classified 92.034512
frame start process 92.1783572
object detected 92.1808986
face detected 92.7259234
face classified 92.7900223
frame start process 92.9463574
object detected 92.9481586
face detected 93.5167158
face classified 93.577102
frame start process 93.7463461
object detected 93.7487274
face detected 94.3033759
face classified 94.36

face detected 140.7011635
face classified 140.762648
frame start process 140.8404236
object detected 140.8423297
face detected 141.4601383
face classified 141.52706
frame start process 141.6666415
object detected 141.668838
face detected 142.2341474
face classified 142.2953015
frame start process 142.3744044
object detected 142.3763816
face detected 142.9146708
face classified 142.9778122
frame start process 143.0316779
object detected 143.0333178
face detected 143.5962446
face classified 143.6805268
frame start process 143.7352428
object detected 143.737319
face detected 144.5640035
face classified 144.6370793
frame start process 144.7223609
object detected 144.7243153
face detected 145.4933094
face classified 145.5593831
frame start process 145.6181389
object detected 145.6199906
face detected 146.2107367
face classified 146.2805463
frame start process 146.3356272
object detected 146.3376811
face detected 146.934049
face detected 146.9535095
face classified 147.1258991
frame start pr

frame start process 198.962202
object detected 198.9643392
face detected 199.5350943
face classified 199.5999338
frame start process 199.6550601
object detected 199.6570268
face detected 200.2715119
face classified 200.3345845
frame start process 200.4394739
object detected 200.4413245
face detected 200.9947763
face detected 201.0154842
face classified 201.0777848
frame start process 201.1382169
object detected 201.1396376
face detected 201.7325111
face classified 201.8079503
face detected 201.8277543
frame start process 201.855645
object detected 201.8575047
face detected 202.4626273
face classified 202.5248581
frame start process 202.5540299
object detected 202.5558621
face detected 203.147656
face classified 203.214067
frame start process 203.3623263
object detected 203.3645632
face detected 203.9014514
face classified 203.970783
frame start process 204.0983109
object detected 204.101054
face detected 204.6751289
frame start process 204.7543501
object detected 204.7562999
face detec

frame start process 268.9482101
object detected 268.9504155
frame start process 269.7146672
object detected 269.7168398
frame start process 270.6422694
object detected 270.6446019
frame start process 271.5924765
object detected 271.5955139
frame start process 272.8504513
object detected 272.8541617
frame start process 273.8102916
object detected 273.8126344
frame start process 274.5462829
object detected 274.5484283
frame start process 275.3462379
object detected 275.3484708
frame start process 276.0342458
object detected 276.0364896
frame start process 276.7383648
object detected 276.7406124
frame start process 277.4742454
object detected 277.4764928
frame start process 278.1783197
object detected 278.180594
frame start process 278.8663114
object detected 278.8685116
frame start process 279.602327
object detected 279.6045311
face detected 280.2916844
frame start process 280.4342793
object detected 280.4362372
frame start process 281.1382355
object detected 281.1404744
frame start proc

frame start process 369.7407129
object detected 369.7428187
frame start process 370.5617808
object detected 370.5644048
frame start process 371.3466867
object detected 371.3482113
frame start process 372.0502696
object detected 372.0524291
frame start process 372.8183263
object detected 372.8206059
frame start process 373.5063082
object detected 373.5085262
frame start process 374.2103638
object detected 374.2134754
frame start process 375.0103116
object detected 375.0125872
frame start process 375.7142313
object detected 375.7165059
frame start process 376.4341441
object detected 376.4358501
frame start process 377.1383196
object detected 377.1405433
frame start process 377.842278
object detected 377.8445694
frame start process 378.6422715
object detected 378.6446164
frame start process 379.3302784
object detected 379.3321054
frame start process 380.0663056
object detected 380.0685549
frame start process 380.7704709
object detected 380.7727374
frame start process 381.5062721
object de

...

face detected 1348.8912631
face classified 1348.9591288
frame start process 1349.1062437
object detected 1349.1084734
face detected 1349.7373739
face classified 1349.8014543
frame start process 1349.9382933
object detected 1349.9405027
face detected 1350.5380756
face classified 1350.6044742
frame start process 1350.7062814
object detected 1350.708424
face detected 1351.3853529
face classified 1351.448886
frame start process 1351.506295
object detected 1351.5079608
face detected 1352.1731386
face classified 1352.2545649
frame start process 1352.3467045
object detected 1352.3488199
face detected 1353.2002859
face classified 1353.2664918
frame start process 1353.3724369
object detected 1353.3755168
face detected 1354.1679126
face classified 1354.2522187
frame start process 1354.2819213
object detected 1354.2837855
face detected 1354.9835515
face classified 1355.0536495
frame start process 1355.1222913
object detected 1355.1242974
face detected 1355.8220536
face classified 1355.8876381
fra

frame start process 1414.7222954
object detected 1414.7245002
frame start process 1415.4319075
object detected 1415.4341047
frame start process 1416.1625103
object detected 1416.1647668
frame start process 1416.8664048
object detected 1416.8686256
frame start process 1417.6182547
object detected 1417.6204049
frame start process 1418.4183006
object detected 1418.4205246
frame start process 1419.1223411
object detected 1419.12464
frame start process 1419.8585088
object detected 1419.8607759
frame start process 1420.5571421
object detected 1420.5596625
frame start process 1421.2552254
object detected 1421.2572092
frame start process 1422.0183425
object detected 1422.0205588
frame start process 1422.7862775
object detected 1422.7886183
frame start process 1423.5063204
object detected 1423.508581
frame start process 1424.3863482
object detected 1424.3885069
frame start process 1425.3783819
object detected 1425.3806156
frame start process 1426.1463651
object detected 1426.1486523
frame start

frame start process 1511.3304347
object detected 1511.3327012
frame start process 1512.0343385
object detected 1512.0365881
frame start process 1512.7703779
object detected 1512.7726148
frame start process 1513.5063298
object detected 1513.5085077
frame start process 1514.2743187
object detected 1514.2766976
frame start process 1514.9623691
object detected 1514.9645588
frame start process 1515.6983591
object detected 1515.7005867
frame start process 1516.3991392
object detected 1516.401238
frame start process 1517.1548853
object detected 1517.1572296
frame start process 1517.8584686
object detected 1517.8608003
frame start process 1518.594284
object detected 1518.5965021
frame start process 1519.282296
object detected 1519.2845132
frame start process 1520.0546001
object detected 1520.0567258
frame start process 1521.0586139
object detected 1521.0608479
frame start process 1521.954301
object detected 1521.9564801
frame start process 1522.6584894
object detected 1522.6615661
frame start 

frame start process 1611.1063263
object detected 1611.1085055
frame start process 1611.8422987
object detected 1611.8445973
frame start process 1612.5623099
object detected 1612.5642846
frame start process 1613.2520546
object detected 1613.2543227
frame start process 1613.9703012
object detected 1613.9725075
frame start process 1614.6583138
object detected 1614.6609125
frame start process 1615.3943309
object detected 1615.3966988
frame start process 1616.2265768
object detected 1616.2288665
frame start process 1617.2344107
object detected 1617.2366933
frame start process 1618.0343269
object detected 1618.0365399
frame start process 1618.8127961
object detected 1618.8150375
frame start process 1619.5542762
object detected 1619.556422
frame start process 1620.2584846
object detected 1620.2607039
frame start process 1620.9622841
object detected 1620.9644976
frame start process 1621.6505672
object detected 1621.6529304
frame start process 1622.4183589
object detected 1622.4206166
frame sta

face detected 1702.4817101
frame start process 1702.626397
object detected 1702.6283481
face detected 1703.267167
Done !
